In [2]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
import numpy as np
import pandas as pd


def target_mean_v1(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  for i in range(data.shape[0]):
    groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
    result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
  return result

def target_mean_v2(data, y_name, x_name):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data.loc[i, x_name] not in value_dict.keys():
      value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] = 1
    else:
      value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
      count_dict[data.loc[i, x_name]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
  return result



y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data_pd = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

In [50]:
%%timeit
result_1 = target_mean_v1(data_pd, 'y', 'x')

1 loop, best of 3: 25.6 s per loop


In [51]:
%%timeit
result_2 = target_mean_v2(data_pd, 'y', 'x')

1 loop, best of 3: 264 ms per loop


In [71]:
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = np.concatenate([y, x], axis=1)

In [5]:
%load_ext Cython

In [26]:
%%cython -a

import numpy as np
cimport numpy as cnp
cpdef target_mean_v3(cnp.ndarray data):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data[i][1] not in value_dict:
      value_dict[data[i][1]] = data[i][0]
      count_dict[data[i][1]] = 1
    else:
      value_dict[data[i][1]] += data[i][0]
      count_dict[data[i][1]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data[i][1]] - data[i][0]) / (count_dict[data[i][1]] - 1)
  return result  

In [27]:
%%timeit
result_3 = target_mean_v3(data)

100 loops, best of 3: 12.9 ms per loop


In [29]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4(cnp.ndarray data):
  result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i in range(data.shape[0]):
    if data[i][1] not in value_dict:
      value_dict[data[i][1]] = data[i][0]
      count_dict[data[i][1]] = 1
    else:
      value_dict[data[i][1]] += data[i][0]
      count_dict[data[i][1]] += 1
  for i in range(data.shape[0]):
    result[i] = (value_dict[data[i][1]] - data[i][0]) / (count_dict[data[i][1]] - 1)
  return result  

In [30]:
%%timeit
result_4 = target_mean_v4(data)

100 loops, best of 3: 12.8 ms per loop


In [77]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v5(cnp.ndarray data):
  cdef int i, j
  cdef double[:] result = np.zeros(data.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i from 0<=i<data.shape[0]:
    if data[i][1] not in value_dict:
      value_dict[data[i][1]] = data[i][0]
      count_dict[data[i][1]] = 1
    else:
      value_dict[data[i][1]] += data[i][0]
      count_dict[data[i][1]] += 1
  for j from 0<=j<data.shape[0]:
    result[j] = (value_dict[data[j][1]] - data[j][0]) / (count_dict[data[j][1]] - 1)
  return result

In [75]:
%%timeit
result_5 = target_mean_v5(data)

100 loops, best of 3: 12.9 ms per loop


In [109]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v6(cnp.ndarray data):
  # cdef cnp.ndarray[long, ndim=2, mode='fortran'] cdata = np.asfortranarray(data)
  cdef cnp.ndarray[long, ndim=2, mode='c'] cdata = np.ascontiguousarray(data)
  cdef int i, j
  cdef double[:] result = np.zeros(cdata.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i from 0<=i<cdata.shape[0]:
    if cdata[i][1] not in value_dict:
      value_dict[cdata[i][1]] = cdata[i][0]
      count_dict[cdata[i][1]] = 1
    else:
      value_dict[cdata[i][1]] += cdata[i][0]
      count_dict[cdata[i][1]] += 1
  for j from 0<=j<cdata.shape[0]:
    result[j] = (value_dict[cdata[j][1]] - cdata[j][0]) / (count_dict[cdata[j][1]] - 1)
  return result

In [110]:
%%timeit
result_6 = target_mean_v6(data)

100 loops, best of 3: 12.8 ms per loop


In [9]:
%%cython -a

import numpy as np
cimport numpy as cnp
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v7(cnp.ndarray[int] data_y, cnp.ndarray[int] data_x):
  cdef int i, j
  cdef double[:] result = np.zeros(data_y.shape[0])
  value_dict = dict()
  count_dict = dict()
  for i from 0<=i<data_y.shape[0]:
    if data_x[i] not in value_dict:
      value_dict[data_x[i]] = data_y[i]
      count_dict[data_x[i]] = 1
    else:
      value_dict[data_x[i]] += data_y[i]
      count_dict[data_x[i]] += 1
  for j from 0<=j<data_y.shape[0]:
    result[j] = (value_dict[data_x[j]] - data_y[j]) / (count_dict[data_x[j]] - 1)
  return result

In [10]:
data_y = np.random.randint(2, size=(5000,), dtype=np.int32)
data_x = np.random.randint(10, size=(5000,), dtype=np.int32)

In [19]:
%%timeit
result_7 = target_mean_v7(data_y, data_x)

1000 loops, best of 3: 1.11 ms per loop


In [16]:
%%cython --cplus

import numpy as np
cimport numpy as cnp
cimport cython
from libcpp.map cimport map
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v8(cnp.ndarray[int] data_y, cnp.ndarray[int] data_x):
  cdef int i, j
  cdef double[:] result = np.zeros(data_y.shape[0])
  cdef map[int, int] value_dict
  cdef map[int, int] count_dict 
  for i in prange(data_y.shape[0], nogil=True, num_threads=4):
    if value_dict.count(data_x[i]):
      value_dict[data_x[i]] += data_y[i]
      count_dict[data_x[i]] += 1
    else:
      value_dict[data_x[i]] = data_y[i]
      count_dict[data_x[i]] = 1
      
  for j in prange(data_y.shape[0], nogil=True, num_threads=4):
    result[j] = (value_dict[data_x[j]] - data_y[j]) / (count_dict[data_x[j]] - 1)
  return result

In [17]:
%%timeit
result_8 = target_mean_v8(data_y, data_x)

1000 loops, best of 3: 413 µs per loop
